# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.
- Run the create_tables.py
- Run the cells below to (not the deleting of database)
- Verify with the ipython-sql notebook

In [1]:
import json
import os
import glob
from datetime import datetime
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
# https://docs.python.org/3.8/library/os.html#os.walk
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [3]:
# Working directory
%pwd

'D:\\PycharmProjects\\udacity_dataengineering\\RDBMS\\Postgres_Data_Modeling'

In [4]:
# Changing the working directory
#%cd "D:\PycharmProjects\udacity_dataengineering\song_data"
song_files_path = "D:\\PycharmProjects\\udacity_dataengineering\\RDBMS\\Postgres_Data_Modeling\\data\\song_data"

In [5]:
# Iterating over the directory where song log files are located
song_files = get_files(song_files_path)
print(len(song_files))
song_files[0]

71


'D:\\PycharmProjects\\udacity_dataengineering\\RDBMS\\Postgres_Data_Modeling\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json'

In [6]:
# Getting the first file for inserting
filepath = song_files[0]

In [24]:
# Initiating an empty dataframe and appending the results of each song log file as a dataframe
song_df = pd.DataFrame(data=None, columns=['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude','artist_location',
                                'artist_name', 'song_id', 'title', 'duration', 'year'])
# Reading first json file into dataframe
for file in song_files:
    file_df = pd.read_json(file, lines = True)
    song_df = song_df.append(file_df, ignore_index=True)

song_df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
1,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
2,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
3,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
4,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [25]:
song_df.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [26]:
# Extracting the columns and a single row values in a list
song_data_one_line =song_df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data_one_line

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [27]:
# connect to sparkify database
conn = psycopg2.connect(database='sparkifydb', user='cta_admin', password="chicago", \
                        host="127.0.0.1", port="5432", sslmode="disable", gssencmode="disable")
conn.set_session(autocommit=True)
cur = conn.cursor()

In [33]:
# Iterating over all rows of songs_df only getting columns needed for songs table
for index, row in song_df[['song_id', 'title', 'artist_id', 'year', 'duration']].iterrows():
    song_data = row.values
    cur.execute(song_table_insert, song_data)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [34]:
song_df.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [35]:
# obtaining the columns and values to a list
artist_data_one_song = song_df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data_one_song

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [36]:
# Iterating over all rows of songs_df only getting columns needed for artists table
for index, row in song_df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].iterrows():
    artist_data = row.values
    cur.execute(artist_table_insert, artist_data)


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
%pwd

'D:\\PycharmProjects\\udacity_dataengineering\\RDBMS\\Postgres_Data_Modeling'

In [37]:
# Iterating over the directory where log files are located
log_files_path = "D:\\PycharmProjects\\udacity_dataengineering\\RDBMS\\Postgres_Data_Modeling\\data\\log_data"
log_files = get_files(log_files_path)
len(log_files)

30

In [38]:
# taking one file
filepath = log_files[0]
filepath

'D:\\PycharmProjects\\udacity_dataengineering\\RDBMS\\Postgres_Data_Modeling\\log_data\\2018\\11\\2018-11-01-events.json'

In [39]:
# Initiating an empty dataframe and appending the results of each log file as a dataframe
log_df = pd.DataFrame(data=None, columns=['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'])
# Reading first json file into dataframe
for file in log_files:
    file_df = pd.read_json(file, lines = True)
    log_df = log_df.append(file_df, ignore_index=True)

log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by page == `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [40]:
log_df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [41]:
# Filtering next song patters
log_df = log_df[log_df['page'] == 'NextSong']
print('The length of the log data with page == NextSong is {}'.format(len(log_df)))
log_df.head()

The length of the log data with page == NextSong is 6820


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [42]:
# Sample convering time in milliseconds to datetime with year, month, day, hour, minute, second, microsecond
ts_sample = datetime.fromtimestamp(log_df['ts'].iloc[0]/1000.0)
ts_sample

datetime.datetime(2018, 11, 1, 14, 1, 46, 796000)

In [43]:
# Obtaining separate time segments
print(f"Year: {ts_sample.year}")
print(f"Month: {ts_sample.month}")
print(f"Day: {ts_sample.day}")
print(f"Hours: {ts_sample.hour}")
print(f"Minutes: {ts_sample.minute}")
print(f"Seconds: {ts_sample.second}")
print(f"Milliseconds: {ts_sample.microsecond}")

Year: 2018
Month: 11
Day: 1
Hours: 14
Minutes: 1
Seconds: 46
Milliseconds: 796000


In [44]:
# Converting ts to datetime and extracting timeframes
log_df['ts'] = log_df['ts'].apply(lambda x: datetime.fromtimestamp(x/1000.0))
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,2018-11-01 14:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,2018-11-01 14:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,2018-11-01 14:08:16.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,2018-11-01 14:11:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,2018-11-01 14:17:33.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [45]:
# Obtaining the week of the year
log_df['ts'].iloc[0].isocalendar()[1]

44

In [46]:
# Adding columns by separating timeframes
log_df['hour'] = log_df['ts'].apply(lambda x: x.hour)
log_df['day'] = log_df['ts'].apply(lambda x: x.day)
log_df['week'] = log_df['ts'].apply(lambda x: x.isocalendar()[1])
log_df['month'] = log_df['ts'].apply(lambda x: x.month)
log_df['year'] = log_df['ts'].apply(lambda x: x.year)
log_df['weekday'] = log_df['ts'].apply(lambda x: x.weekday())
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,status,ts,userAgent,userId,hour,day,week,month,year,weekday
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,200,2018-11-01 14:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,14,1,44,11,2018,3
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,200,2018-11-01 14:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,14,1,44,11,2018,3
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,200,2018-11-01 14:08:16.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,14,1,44,11,2018,3
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,200,2018-11-01 14:11:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,14,1,44,11,2018,3
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,200,2018-11-01 14:17:33.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,14,1,44,11,2018,3


In [47]:
# Extract timestamp, hour, day, week of year, month, year, and weekday from the `ts` column
time_df = log_df[['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']]
time_df.head()

,ts,hour,day,week,month,year,weekday
2,2018-11-01 14:01:46.796,14,1,44,11,2018,3
4,2018-11-01 14:05:52.796,14,1,44,11,2018,3
5,2018-11-01 14:08:16.796,14,1,44,11,2018,3
6,2018-11-01 14:11:13.796,14,1,44,11,2018,3
7,2018-11-01 14:17:33.796,14,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [49]:
# Iterating over all rows of log_df only getting columns needed for time table
for index, row in log_df[['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']].iterrows():
    time_data = row.values
    cur.execute(time_table_insert, time_data)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [28]:
log_df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId', 'hour',
       'day', 'week', 'month', 'year', 'weekday'],
      dtype='object')

In [29]:
# extracting user information
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [50]:
# Iterating over all rows of log_df only getting columns needed for user table
for index, row in log_df[['userId', 'firstName', 'lastName', 'gender', 'level']].iterrows():
    user_data = row.values
    cur.execute(user_table_insert, user_data)

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

### Joining Data from tables
- Perform a join using the song title, artist name, and duration to match the song id and artist id

sparkifydb=# SELECT s.song_id, s.title, a.artist_id, a.artist_name, a.artist_location FROM songs s JOIN artists a ON s.artist_id=a.artist_id WHERE s.title = 'I Didn''t Mean To' AND a.artist_name= 'Casual';
      song_id       |      title       |     artist_id      | artist_name | artist_location
--------------------+------------------+--------------------+-------------+-----------------
 SOMZWCG12A8C13C480 | I Didn't Mean To | ARD7TVE1187B99BFB1 | Casual      | California - LA


In [31]:
log_df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId', 'hour',
       'day', 'week', 'month', 'year', 'weekday'],
      dtype='object')

In [51]:
# Testing the song_select function with a known values
'''
[('SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  'Casual',
  Decimal('218.93179'))]
'''
#cur.execute(song_select, ['I Didn''t Mean To', 'Casual', 218.93179])
cur.execute(song_select, ["I Didn't Mean To", 'Casual', 218.93179])
cur.fetchall()

[('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1')]

In [52]:
# Iterating across all log rows to join data from other tables
for index, row in log_df.iterrows():
    '''
    creating a temporary list of song play data and get songid and artistid from song and artist tables
    '''
    song_play_data = []
    try:
        cur.execute(song_select, [row.song, row.artist, row.length])
        results = cur.fetchone()
        #print(results)
        if results:
            song_id, artist_id = results
        else:
            song_id, artist_id = None, None
    except Exception as e:
        print(e)

    song_play_data.append([row.ts, row.userId, row.level, song_id, artist_id, row.sessionId, row.location, row.userAgent])
    cur.execute(song_play_table_insert, song_play_data[0])
    #print(song_play_data)


#cur.executemany(song_play_table_insert, song_play_data)


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [53]:
cur.close()
conn.close()